In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
import random
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.label = df.label.astype("object")
df.station_id = df.station_id.astype("object")
df.line_id = df.line_id.astype("object")
df.ticket_id = df.ticket_id.astype("object")
df.id = df.id.astype("object")

In [4]:
del df['id']

In [5]:
df.entrance_id = df.entrance_id.astype("object")

In [6]:
df['year'] = pd.DatetimeIndex(df['pass_dttm']).year
df['month'] = pd.DatetimeIndex(df['pass_dttm']).month
df['day'] = pd.DatetimeIndex(df['pass_dttm']).day
df['weekday'] = pd.DatetimeIndex(df['pass_dttm']).weekday
df['hour'] = pd.DatetimeIndex(df['pass_dttm']).hour
df['minute'] = pd.DatetimeIndex(df['pass_dttm']).minute

In [7]:
del df['pass_dttm']

In [8]:
df.weekday = df.weekday.astype("object")

In [9]:
df.head()

,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,time_to_under,label,year,month,day,weekday,hour,minute
0,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,216.316667,8001,2022,9,12,0,5,0
1,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,648.183333,9011,2022,9,12,0,5,0
2,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,865.333333,7022,2022,9,12,0,5,0
3,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,1048.233333,2022,2022,9,12,0,5,1
4,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,965.600000,2017,2022,9,12,0,5,2


In [10]:
del df['station_nm']
del df['line_nm']
del df['entrance_nm']
del df['ticket_id']
del df['minute']

In [11]:
features = ["year", "month", "day", "hour"]
target = ["time_to_under", "label"]
category = []
for i in df.columns:
    if i not in features and i not in target:
        category.append(i)

In [12]:
'''
for i in category:
    print(i)
    df[i] = pd.Categorical(df[i])
    df[i].astype('category').cat.codes
    df[i] = df[i].cat.codes
'''

"\nfor i in category:\n    print(i)\n    df[i] = pd.Categorical(df[i])\n    df[i].astype('category').cat.codes\n    df[i] = df[i].cat.codes\n"

In [13]:
category

['ticket_type_nm', 'entrance_id', 'station_id', 'line_id', 'weekday']

In [14]:
from category_encoders.binary import BinaryEncoder
d = pd.DataFrame()
ct = []
binens = []
for i in category:
    print(i)
    bn = BinaryEncoder()
    bn.fit(df[i].values)
    binens.append(bn)
    k = bn.transform(df[i].values)
    l  = []
    for j in k.columns:
        l.append(str(i)+str(j))
    ct.extend(l)
    k.columns = l
    d = pd.concat([d, k], axis=1)

ticket_type_nm
entrance_id
station_id
line_id
weekday


In [15]:
feat = features.copy()
feat.extend(target)

In [16]:
df = df[feat]

In [17]:
df = pd.concat([df, d], axis=1)

In [18]:
np.random.seed(77)
val_ds = pd.DataFrame()
train_ds = pd.DataFrame()
k = 0
l = len(df.label.unique())
for label in df.label.unique():
    k+=1
    if k%int(l/25)==0: print(k)
    #n = round(0.2*len(df.loc[df['label']==label]))
    n = 5
    ids = random.sample(list(df.loc[df.label == label].index), n)
    val_ds = pd.concat([val_ds, df.loc[ids]])
    train_ids = list(set(df.loc[df.label == label].index) - set(ids))
    train_ds = pd.concat([train_ds, df.loc[train_ids]])

11
22
33
44
55
66
77
88
99
110
121
132
143
154
165
176
187
198
209
220
231
242
253
264
275


In [19]:
val_ds.shape[0]/train_ds.shape[0]

0.0012664721683563669

In [20]:
train_ds['label'] = train_ds['label'].astype('int64')

In [21]:
train = train_ds.copy()
y = train['time_to_under']
del train['time_to_under']
del train['label']
X = train

In [22]:
X.head(2)

,year,month,day,hour,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,ticket_type_nm0_5,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,12,5,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,1
851969,2022,9,17,6,0,1,0,0,0,1,...,1,1,0,0,0,1,0,1,1,0


In [35]:
model = CatBoostRegressor(iterations=500,
                              cat_features = ct,
                             task_type = 'GPU')

In [42]:
model.fit(X,y)

Learning rate set to 0.360664
0:	learn: 197.3402914	total: 19.6ms	remaining: 1.94s
1:	learn: 180.2989964	total: 35.6ms	remaining: 1.74s
2:	learn: 171.2218471	total: 52.6ms	remaining: 1.7s
3:	learn: 167.4061749	total: 70ms	remaining: 1.68s
4:	learn: 165.1321068	total: 87.7ms	remaining: 1.67s
5:	learn: 163.0583516	total: 105ms	remaining: 1.65s
6:	learn: 162.1438961	total: 125ms	remaining: 1.66s
7:	learn: 161.8129175	total: 140ms	remaining: 1.61s
8:	learn: 161.6437964	total: 156ms	remaining: 1.58s
9:	learn: 161.4069386	total: 172ms	remaining: 1.55s
10:	learn: 161.1932470	total: 190ms	remaining: 1.54s
11:	learn: 160.9653071	total: 206ms	remaining: 1.51s
12:	learn: 160.6662558	total: 223ms	remaining: 1.49s
13:	learn: 160.5876242	total: 239ms	remaining: 1.47s
14:	learn: 160.4573877	total: 256ms	remaining: 1.45s
15:	learn: 160.4315632	total: 271ms	remaining: 1.42s
16:	learn: 160.3218228	total: 287ms	remaining: 1.4s
17:	learn: 160.2774553	total: 303ms	remaining: 1.38s
18:	learn: 160.1782934	to

54:	learn: 159.9930544	total: 819ms	remaining: 670ms
55:	learn: 159.9830173	total: 834ms	remaining: 655ms
56:	learn: 159.9773047	total: 851ms	remaining: 642ms
57:	learn: 159.9526306	total: 867ms	remaining: 628ms
58:	learn: 159.9392418	total: 883ms	remaining: 613ms
59:	learn: 159.9331897	total: 897ms	remaining: 598ms
60:	learn: 159.9216430	total: 912ms	remaining: 583ms
61:	learn: 159.9098164	total: 925ms	remaining: 567ms
62:	learn: 159.8967767	total: 942ms	remaining: 553ms
63:	learn: 159.8887686	total: 958ms	remaining: 539ms
64:	learn: 159.8755656	total: 973ms	remaining: 524ms
65:	learn: 159.8690702	total: 990ms	remaining: 510ms
66:	learn: 159.8643454	total: 1.01s	remaining: 497ms
67:	learn: 159.8540724	total: 1.02s	remaining: 482ms
68:	learn: 159.8458198	total: 1.04s	remaining: 468ms
69:	learn: 159.8188174	total: 1.05s	remaining: 452ms
70:	learn: 159.8047118	total: 1.07s	remaining: 437ms
71:	learn: 159.8003894	total: 1.08s	remaining: 422ms
72:	learn: 159.7864659	total: 1.1s	remaining: 

13:	learn: 160.2022910	total: 210ms	remaining: 1.29s
14:	learn: 160.0781608	total: 226ms	remaining: 1.28s
15:	learn: 160.0183279	total: 239ms	remaining: 1.25s
16:	learn: 159.9730200	total: 254ms	remaining: 1.24s
17:	learn: 159.9182752	total: 266ms	remaining: 1.21s
18:	learn: 159.8727552	total: 285ms	remaining: 1.21s
19:	learn: 159.8463674	total: 303ms	remaining: 1.21s
20:	learn: 159.7964899	total: 325ms	remaining: 1.22s
21:	learn: 159.7708468	total: 346ms	remaining: 1.23s
22:	learn: 159.7105163	total: 360ms	remaining: 1.2s
23:	learn: 159.6882580	total: 374ms	remaining: 1.19s
24:	learn: 159.6708523	total: 388ms	remaining: 1.17s
25:	learn: 159.6017257	total: 403ms	remaining: 1.15s
26:	learn: 159.5747262	total: 417ms	remaining: 1.13s
27:	learn: 159.5579560	total: 430ms	remaining: 1.11s
28:	learn: 159.5322083	total: 445ms	remaining: 1.09s
29:	learn: 159.4986130	total: 458ms	remaining: 1.07s
30:	learn: 159.4615261	total: 473ms	remaining: 1.05s
31:	learn: 159.4232444	total: 486ms	remaining: 

70:	learn: 158.1114474	total: 1.01s	remaining: 414ms
71:	learn: 158.0979919	total: 1.03s	remaining: 400ms
72:	learn: 158.0871657	total: 1.04s	remaining: 385ms
73:	learn: 158.0773494	total: 1.06s	remaining: 372ms
74:	learn: 158.0706687	total: 1.07s	remaining: 358ms
75:	learn: 158.0657044	total: 1.09s	remaining: 344ms
76:	learn: 158.0592833	total: 1.1s	remaining: 329ms
77:	learn: 158.0465814	total: 1.12s	remaining: 315ms
78:	learn: 158.0382939	total: 1.13s	remaining: 300ms
79:	learn: 158.0271217	total: 1.14s	remaining: 286ms
80:	learn: 158.0165434	total: 1.16s	remaining: 271ms
81:	learn: 158.0111388	total: 1.17s	remaining: 257ms
82:	learn: 157.9967231	total: 1.18s	remaining: 242ms
83:	learn: 157.9889682	total: 1.2s	remaining: 228ms
84:	learn: 157.9833544	total: 1.21s	remaining: 213ms
85:	learn: 157.9755395	total: 1.22s	remaining: 199ms
86:	learn: 157.9656121	total: 1.24s	remaining: 185ms
87:	learn: 157.9574914	total: 1.25s	remaining: 171ms
88:	learn: 157.9472729	total: 1.27s	remaining: 1

28:	learn: 159.7704960	total: 525ms	remaining: 8.53s
29:	learn: 159.7627199	total: 539ms	remaining: 8.45s
30:	learn: 159.7554023	total: 553ms	remaining: 8.37s
31:	learn: 159.7503844	total: 567ms	remaining: 8.3s
32:	learn: 159.7461018	total: 581ms	remaining: 8.22s
33:	learn: 159.7426250	total: 595ms	remaining: 8.15s
34:	learn: 159.7394423	total: 608ms	remaining: 8.08s
35:	learn: 159.7368184	total: 622ms	remaining: 8.02s
36:	learn: 159.7338238	total: 636ms	remaining: 7.95s
37:	learn: 159.7326943	total: 649ms	remaining: 7.89s
38:	learn: 159.7302232	total: 662ms	remaining: 7.83s
39:	learn: 159.7284170	total: 675ms	remaining: 7.77s
40:	learn: 159.7262225	total: 689ms	remaining: 7.71s
41:	learn: 159.7244574	total: 702ms	remaining: 7.66s
42:	learn: 159.7234513	total: 716ms	remaining: 7.61s
43:	learn: 159.7223510	total: 730ms	remaining: 7.57s
44:	learn: 159.7207389	total: 744ms	remaining: 7.52s
45:	learn: 159.7191091	total: 757ms	remaining: 7.47s
46:	learn: 159.7175969	total: 770ms	remaining: 

183:	learn: 159.6154024	total: 2.74s	remaining: 4.7s
184:	learn: 159.6147960	total: 2.75s	remaining: 4.69s
185:	learn: 159.6145664	total: 2.77s	remaining: 4.67s
186:	learn: 159.6141425	total: 2.78s	remaining: 4.65s
187:	learn: 159.6133947	total: 2.79s	remaining: 4.64s
188:	learn: 159.6130591	total: 2.81s	remaining: 4.62s
189:	learn: 159.6127000	total: 2.83s	remaining: 4.61s
190:	learn: 159.6120582	total: 2.84s	remaining: 4.59s
191:	learn: 159.6115578	total: 2.85s	remaining: 4.58s
192:	learn: 159.6111162	total: 2.87s	remaining: 4.56s
193:	learn: 159.6104567	total: 2.88s	remaining: 4.54s
194:	learn: 159.6100152	total: 2.9s	remaining: 4.53s
195:	learn: 159.6097502	total: 2.91s	remaining: 4.51s
196:	learn: 159.6094028	total: 2.92s	remaining: 4.49s
197:	learn: 159.6091143	total: 2.94s	remaining: 4.48s
198:	learn: 159.6087316	total: 2.95s	remaining: 4.46s
199:	learn: 159.6082782	total: 2.96s	remaining: 4.45s
200:	learn: 159.6078602	total: 2.98s	remaining: 4.43s
201:	learn: 159.6075422	total:

336:	learn: 159.5570746	total: 4.92s	remaining: 2.38s
337:	learn: 159.5566565	total: 4.94s	remaining: 2.37s
338:	learn: 159.5562088	total: 4.95s	remaining: 2.35s
339:	learn: 159.5560910	total: 4.96s	remaining: 2.34s
340:	learn: 159.5559261	total: 4.98s	remaining: 2.32s
341:	learn: 159.5555904	total: 4.99s	remaining: 2.31s
342:	learn: 159.5553784	total: 5.01s	remaining: 2.29s
343:	learn: 159.5549072	total: 5.02s	remaining: 2.28s
344:	learn: 159.5547128	total: 5.04s	remaining: 2.26s
345:	learn: 159.5539530	total: 5.05s	remaining: 2.25s
346:	learn: 159.5536762	total: 5.07s	remaining: 2.23s
347:	learn: 159.5534759	total: 5.08s	remaining: 2.22s
348:	learn: 159.5531755	total: 5.09s	remaining: 2.2s
349:	learn: 159.5529105	total: 5.11s	remaining: 2.19s
350:	learn: 159.5525277	total: 5.12s	remaining: 2.17s
351:	learn: 159.5523097	total: 5.14s	remaining: 2.16s
352:	learn: 159.5520800	total: 5.15s	remaining: 2.15s
353:	learn: 159.5516265	total: 5.17s	remaining: 2.13s
354:	learn: 159.5512966	total

498:	learn: 159.5104324	total: 7.29s	remaining: 14.6ms
499:	learn: 159.5101084	total: 7.3s	remaining: 0us


StackingRegressor(estimators=[('cb',
                               <catboost.core.CatBoostRegressor object at 0x000002004F2F28B0>),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...))],
                  final_estimator=<catboost.core.CatBoostRegressor object at 0x000002004F31BBE0>)

In [37]:
# посмотрим на важность признаков
imp = pd.DataFrame({
    'feature_name':X.columns,
    'feature_score': model.feature_importances_
}).sort_values('feature_score', ascending=False).reset_index(drop=True)

imp.to_numpy()

array([['hour', 52.724425726784766],
       ['ticket_type_nm0_5', 12.450776920552384],
       ['ticket_type_nm0_3', 8.681647238087155],
       ['ticket_type_nm0_4', 8.64560994098086],
       ['ticket_type_nm0_1', 5.881054906018065],
       ['day', 3.804142916957648],
       ['ticket_type_nm0_2', 2.266111151390053],
       ['line_id0_2', 0.4710581475858633],
       ['line_id0_4', 0.4436631569450717],
       ['ticket_type_nm0_0', 0.4374588542101108],
       ['line_id0_1', 0.42085917646286375],
       ['line_id0_3', 0.404125321666146],
       ['station_id0_4', 0.3498304429559958],
       ['station_id0_5', 0.33068711272025386],
       ['station_id0_6', 0.2702123402642262],
       ['entrance_id0_0', 0.26675451345044604],
       ['station_id0_7', 0.23467166152435714],
       ['station_id0_3', 0.19776459186048503],
       ['station_id0_8', 0.18042430075699586],
       ['entrance_id0_1', 0.17448271608373928],
       ['entrance_id0_6', 0.16549796176804302],
       ['entrance_id0_8', 0.158326881

In [43]:
from sklearn.metrics import r2_score
val_ds['label'] = val_ds['label'].astype('int64')
X_test = val_ds.copy()
del X_test['label']
del X_test['time_to_under']
pred = model.predict(X_test)
r2 = r2_score(val_ds.time_to_under.to_list(), pred)
print(model,'\n',r2_score(val_ds.time_to_under.to_list(), pred))

StackingRegressor(estimators=[('cb',
                               <catboost.core.CatBoostRegressor object at 0x000002004F2F28B0>),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=

In [39]:
train = train_ds.copy()
y = train['label']
#del train['time_to_under']
del train['label']
X = train

In [40]:
X.head()

,year,month,day,hour,time_to_under,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,12,5,216.316667,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
851969,2022,9,17,6,697.716667,0,1,0,0,0,...,1,1,0,0,0,1,0,1,1,0
589832,2022,9,15,8,602.350000,0,1,0,0,0,...,0,0,0,1,0,0,1,1,0,0
655383,2022,9,15,13,343.016667,0,1,0,1,1,...,1,0,0,1,0,1,1,1,0,0
688151,2022,9,16,6,771.666667,0,0,1,1,1,...,0,1,0,0,0,1,0,1,0,1


In [41]:
clf = RandomForestClassifier(random_state=77, n_estimators=10)

In [42]:
clf.fit(X.to_numpy(), y.to_numpy())

RandomForestClassifier(n_estimators=10, random_state=77)

In [43]:
clf.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 4.26665178e-02, 1.69171750e-01,
       5.80141126e-01, 3.83361555e-03, 1.84392784e-02, 2.36228792e-02,
       1.76868975e-02, 2.51226730e-02, 3.23810429e-02, 9.68850457e-04,
       1.18408734e-03, 1.66589848e-03, 1.57521509e-03, 1.35511468e-03,
       1.99457473e-03, 1.93927742e-03, 1.97566587e-03, 1.93770746e-03,
       1.20183229e-04, 9.74938482e-04, 1.22024300e-03, 1.19916535e-03,
       1.28990544e-03, 1.25445412e-03, 1.39474367e-03, 1.42521398e-03,
       1.42699805e-03, 6.25068468e-06, 1.57538091e-03, 1.14617583e-03,
       1.18121555e-03, 1.26890068e-03, 1.60955492e-02, 1.85051292e-02,
       2.22533805e-02])

In [44]:
from sklearn.metrics import recall_score
val_ds['label'] = val_ds['label'].astype('int64')
X_test = val_ds.copy()
del X_test['label']
X_test['time_to_under'] = pred
pred1 = clf.predict(X_test)
recall = recall_score(val_ds.label.to_list(), pred1, average="macro", zero_division=0)
print(clf,'\n',recall_score(val_ds.label.to_list(), pred1, average="macro", zero_division=0))

C:\anaconda\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


RandomForestClassifier(n_estimators=10, random_state=77) 
 0.04057971014492753


In [45]:
res = 0.5*r2+0.5*recall
res

0.2857499022917061

In [44]:
#Обучим на всей выборке

In [21]:
train = df.copy()
y = train['time_to_under']
del train['time_to_under']
del train['label']
X = train

In [22]:
X.head(1)

,year,month,day,hour,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,ticket_type_nm0_5,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,12,5,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,1


In [21]:
model = CatBoostRegressor(iterations=500,
                              cat_features = ct,
                             task_type = 'GPU')

In [23]:
from xgboost import XGBRegressor
model = XGBRegressor()

In [24]:
 model.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [23]:
# посмотрим на важность признаков
imp = pd.DataFrame({
    'feature_name':X.columns,
    'feature_score': model.feature_importances_
}).sort_values('feature_score', ascending=False).reset_index(drop=True)

imp.to_numpy()

array([['hour', 47.05504107392876],
       ['ticket_type_nm0_5', 13.472224667242006],
       ['ticket_type_nm0_3', 9.202402581776996],
       ['ticket_type_nm0_1', 8.209116684816983],
       ['ticket_type_nm0_4', 6.866749514145023],
       ['day', 4.158383510953153],
       ['ticket_type_nm0_2', 2.577318183889997],
       ['line_id0_2', 0.5945530629283122],
       ['ticket_type_nm0_0', 0.5844666945920807],
       ['line_id0_3', 0.5621370337457258],
       ['line_id0_1', 0.5533017428836928],
       ['line_id0_4', 0.5439186217430675],
       ['station_id0_7', 0.4501144464756549],
       ['station_id0_5', 0.43777670801407514],
       ['station_id0_3', 0.42300219338248574],
       ['station_id0_6', 0.3802903203521885],
       ['station_id0_4', 0.3577633522499438],
       ['entrance_id0_0', 0.3508104132431871],
       ['station_id0_8', 0.31240650774671236],
       ['entrance_id0_6', 0.30769965553151896],
       ['entrance_id0_3', 0.3042138191051864],
       ['entrance_id0_2', 0.279454981426

In [25]:
from sklearn.metrics import r2_score
val_ds['label'] = val_ds['label'].astype('int64')
X_test = val_ds.copy()
del X_test['label']
del X_test['time_to_under']
pred = model.predict(X_test)
print(pred)
#X = X_test['postmark'].copy()
#X.loc[X==0.0] = 0
#X.loc[X!=0] = 1
#pred = pr(X.to_list())
r2 = r2_score(val_ds.time_to_under.to_list(), pred)
print(model,'\n',r2_score(val_ds.time_to_under.to_list(), pred))

[567.79565 598.4594  680.2964  ... 401.8845  135.56525 496.40994]
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...) 
 0.5365789940403094


In [26]:
train = df.copy()
train['label'] = train['label'].astype('int64')
y = train['label']
#del train['time_to_under']
del train['label']
X = train

In [27]:
clf = RandomForestClassifier(random_state=77, n_estimators=10)

In [28]:
X.head(1)

,year,month,day,hour,time_to_under,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,12,5,216.316667,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1


In [29]:
clf.fit(X.to_numpy(), y.to_numpy())

RandomForestClassifier(n_estimators=10, random_state=77)

In [30]:
clf.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 4.29265434e-02, 1.66837565e-01,
       5.78209689e-01, 3.87340243e-03, 1.86534448e-02, 2.41389378e-02,
       1.82975095e-02, 2.57317110e-02, 3.28948622e-02, 9.65797439e-04,
       1.24248986e-03, 1.83018262e-03, 1.58902122e-03, 1.46876059e-03,
       1.94834999e-03, 1.85295223e-03, 2.05503097e-03, 2.02938596e-03,
       1.22413724e-04, 1.07002821e-03, 1.29872990e-03, 1.43392775e-03,
       1.30504751e-03, 1.35262635e-03, 1.44473465e-03, 1.68913228e-03,
       1.47824476e-03, 9.37515494e-06, 1.56790827e-03, 9.98829925e-04,
       1.13725421e-03, 1.26879325e-03, 1.60472120e-02, 1.87723692e-02,
       2.24577376e-02])

In [31]:
from sklearn.metrics import recall_score
val_ds['label'] = val_ds['label'].astype('int64')
X_test = val_ds.copy()
del X_test['label']
#del X_test['ticket_id']
X_test['time_to_under'] = pred
pred1 = clf.predict(X_test)
print(pred1)
#X = X_test['postmark'].copy()
#X.loc[X==0.0] = 0
#X.loc[X!=0] = 1
#pred = pr(X.to_list())
recall = recall_score(val_ds.label.to_list(), pred1, average="macro", zero_division=0)
print(clf,'\n',recall_score(val_ds.label.to_list(), pred1, average="macro", zero_division=0))

C:\anaconda\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


[ 1022  6015  8006 ...  6006 13006 13001]
RandomForestClassifier(n_estimators=10, random_state=77) 
 0.24420289855072466


In [32]:
res = 0.5*r2+0.5*recall
res

0.390390946295517

In [32]:
#сделаем предсказание на тестовом датасете

In [33]:
test_df = pd.read_csv("test.csv")

In [34]:
test_df.head()

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm
0,1091022,425A13E16226CA01032043111488E0F5,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:03
1,1091023,F94EDC107A4E054786882F9453C1B93F,Пропуск FacePay,184,Молодёжная (Западный),3010,Молодёжная,3,Арбатско-Покровская,2022-09-12 05:00:45
2,1091024,8F13BFBA98A7DD34B11BD2DDCC5CA68E,СК студента 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:52
3,1091025,7B98691636EC418ABF6FEDDE4D7C68FC,КОШЕЛЕК,107,Сокол ( Южн./Вост. ),2018,Сокол,2,Замоскворецкая,2022-09-12 05:00:58
4,1091026,679C55C28CB36FB546FD104502C0B3DB,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:39


In [35]:
test_df.shape

(561890, 10)

In [36]:
test_df.station_id = test_df.station_id.astype("object")
test_df.line_id = test_df.line_id.astype("object")
test_df.ticket_id = test_df.ticket_id.astype("object")
test_df.entrance_id = test_df.entrance_id.astype("object")

In [37]:
test_df['year'] = pd.DatetimeIndex(test_df['pass_dttm']).year
test_df['month'] = pd.DatetimeIndex(test_df['pass_dttm']).month
test_df['day'] = pd.DatetimeIndex(test_df['pass_dttm']).day
test_df['weekday'] = pd.DatetimeIndex(test_df['pass_dttm']).weekday
test_df['hour'] = pd.DatetimeIndex(test_df['pass_dttm']).hour
test_df['minute'] = pd.DatetimeIndex(test_df['pass_dttm']).minute

In [38]:
del test_df['pass_dttm']

In [39]:
test_df.weekday = test_df.weekday.astype("object")

In [40]:
del test_df['station_nm']
del test_df['line_nm']
del test_df['entrance_nm']
del test_df['ticket_id']
del test_df['minute']

In [41]:
from category_encoders.binary import BinaryEncoder
d = pd.DataFrame()
ct = []
#binens = []

for i in range(0, len(category)):
    print(category[i])
    if i != "name_mfi":
        k = binens[i].transform(test_df[category[i]].values)
        l  = []
        for j in k.columns:
            l.append(str(category[i])+str(j))
        ct.extend(l)
        k.columns = l
        d = pd.concat([d, k], axis=1)

ticket_type_nm
entrance_id
station_id
line_id
weekday


In [42]:
test_features = features.copy()
test_features.append("id")

In [43]:
test_df = test_df[test_features]

In [44]:
test_df = pd.concat([test_df, d], axis=1)

In [45]:
test_df.shape

(561890, 37)

In [46]:
X = test_df.copy()

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091021 entries, 0 to 1091020
Data columns (total 38 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   year               1091021 non-null  int64  
 1   month              1091021 non-null  int64  
 2   day                1091021 non-null  int64  
 3   hour               1091021 non-null  int64  
 4   time_to_under      1091021 non-null  float64
 5   label              1091021 non-null  object 
 6   ticket_type_nm0_0  1091021 non-null  int64  
 7   ticket_type_nm0_1  1091021 non-null  int64  
 8   ticket_type_nm0_2  1091021 non-null  int64  
 9   ticket_type_nm0_3  1091021 non-null  int64  
 10  ticket_type_nm0_4  1091021 non-null  int64  
 11  ticket_type_nm0_5  1091021 non-null  int64  
 12  entrance_id0_0     1091021 non-null  int64  
 13  entrance_id0_1     1091021 non-null  int64  
 14  entrance_id0_2     1091021 non-null  int64  
 15  entrance_id0_3     1091021 non-n

In [48]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561890 entries, 0 to 561889
Data columns (total 37 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   year               561890 non-null  int64
 1   month              561890 non-null  int64
 2   day                561890 non-null  int64
 3   hour               561890 non-null  int64
 4   id                 561890 non-null  int64
 5   ticket_type_nm0_0  561890 non-null  int64
 6   ticket_type_nm0_1  561890 non-null  int64
 7   ticket_type_nm0_2  561890 non-null  int64
 8   ticket_type_nm0_3  561890 non-null  int64
 9   ticket_type_nm0_4  561890 non-null  int64
 10  ticket_type_nm0_5  561890 non-null  int64
 11  entrance_id0_0     561890 non-null  int64
 12  entrance_id0_1     561890 non-null  int64
 13  entrance_id0_2     561890 non-null  int64
 14  entrance_id0_3     561890 non-null  int64
 15  entrance_id0_4     561890 non-null  int64
 16  entrance_id0_5     561890 non-null  in

In [49]:
del X['id']

In [50]:
pred = model.predict(X)
print(pred)

[693.7107  487.9439  639.9513  ... 527.30524  70.50696  82.10585]


In [51]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1380 entries, 995678 to 414385
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               1380 non-null   int64  
 1   month              1380 non-null   int64  
 2   day                1380 non-null   int64  
 3   hour               1380 non-null   int64  
 4   time_to_under      1380 non-null   float32
 5   ticket_type_nm0_0  1380 non-null   int64  
 6   ticket_type_nm0_1  1380 non-null   int64  
 7   ticket_type_nm0_2  1380 non-null   int64  
 8   ticket_type_nm0_3  1380 non-null   int64  
 9   ticket_type_nm0_4  1380 non-null   int64  
 10  ticket_type_nm0_5  1380 non-null   int64  
 11  entrance_id0_0     1380 non-null   int64  
 12  entrance_id0_1     1380 non-null   int64  
 13  entrance_id0_2     1380 non-null   int64  
 14  entrance_id0_3     1380 non-null   int64  
 15  entrance_id0_4     1380 non-null   int64  
 16  entrance_id0_5   

In [52]:
X = X[features]
X['time_to_under'] = pred
X = pd.concat([X, test_df[ct]], axis=1)

In [53]:
X.head()

,year,month,day,hour,time_to_under,ticket_type_nm0_0,ticket_type_nm0_1,ticket_type_nm0_2,ticket_type_nm0_3,ticket_type_nm0_4,...,station_id0_7,station_id0_8,line_id0_0,line_id0_1,line_id0_2,line_id0_3,line_id0_4,weekday0_0,weekday0_1,weekday0_2
0,2022,9,12,5,693.710693,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
1,2022,9,12,5,487.943909,0,0,0,0,0,...,1,1,0,0,1,1,0,0,0,1
2,2022,9,12,5,639.951294,0,0,0,1,1,...,1,0,0,0,0,1,0,0,0,1
3,2022,9,12,5,705.138489,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1
4,2022,9,12,5,693.710693,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [54]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561890 entries, 0 to 561889
Data columns (total 37 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   year               561890 non-null  int64  
 1   month              561890 non-null  int64  
 2   day                561890 non-null  int64  
 3   hour               561890 non-null  int64  
 4   time_to_under      561890 non-null  float32
 5   ticket_type_nm0_0  561890 non-null  int64  
 6   ticket_type_nm0_1  561890 non-null  int64  
 7   ticket_type_nm0_2  561890 non-null  int64  
 8   ticket_type_nm0_3  561890 non-null  int64  
 9   ticket_type_nm0_4  561890 non-null  int64  
 10  ticket_type_nm0_5  561890 non-null  int64  
 11  entrance_id0_0     561890 non-null  int64  
 12  entrance_id0_1     561890 non-null  int64  
 13  entrance_id0_2     561890 non-null  int64  
 14  entrance_id0_3     561890 non-null  int64  
 15  entrance_id0_4     561890 non-null  int64  
 16  en

In [55]:
pred1 = clf.predict(X)

C:\anaconda\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [56]:
pred1

array([9010, 3016, 2015, ..., 1020, 6015, 2011], dtype=int64)

In [57]:
sample = pd.read_csv("sample_solution.csv")

In [58]:
sample.head()

,id,time_to_under,label
0,1091022,0,0
1,1091023,0,0
2,1091024,0,0
3,1091025,0,0
4,1091026,0,0


In [59]:
result = pd.DataFrame({"id": test_df.id.to_list(), "time_to_under": pred, "label": pred1})

In [60]:
result.head()

,id,time_to_under,label
0,1091022,693.710693,9010
1,1091023,487.943909,3016
2,1091024,639.951294,2015
3,1091025,705.138489,2005
4,1091026,693.710693,9010


In [61]:
result.to_csv("submission4.csv", index=False)